## This notebook gets the images from a google drive folder and saves them as tensors

In [5]:
import os
import io
import cv2
import tqdm
import torch
import h5py
import gc
import numpy as np

import torchvision.transforms as transforms
from pillow_heif import register_heif_opener
from PIL import Image



from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


To run below code it is required to have cloud access to our google drive

In [2]:
SCOPES = ["https://www.googleapis.com/auth/drive"]

creds = None

if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)


if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
        
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "cloud_client.json", SCOPES)

        creds = flow.run_local_server(port=0)


with open("token.json", "w") as token:
    token.write(creds.to_json())



try:

    # Start Google drive cloud service
    service = build("drive", "v3", credentials=creds)

    # Get list of files matching q query
    # These files will list the folders inside the shared folder
    response = service.files().list(
        q="sharedWithMe and mimeType ='application/vnd.google-apps.folder' and name contains 'Capstone'", # and name contains 'cat'",
        spaces="drive"
    ).execute()
        
    # List of files that matched the q query
    capstone_folders = response.get('files', [])

    # Save here all children folders:
    # random cat, same cat, random dog, same dog
    children_folders = []
    
    if not capstone_folders:
        print('No capstone folder found.')
        
    else:

        for parent_folder in capstone_folders:

            capstone_id = parent_folder["id"]

            # Get the list of all the folders inside the folder parent_folder["id"]
            response = service.files().list(
                q=f"'{capstone_id}' in parents and mimeType='application/vnd.google-apps.folder'", # and name contains 'cat'",
                spaces="drive"
            ).execute()

            child_folders_temp = response.get('files', [])

            if child_folders_temp:
                for child_folder in child_folders_temp:
                    children_folders.append(child_folder)


    animal_files_in_folders = {}
    
    # Read images folder by folder to get file IDs from the following folders:
    # random cat, same cat, random dog, same dog
    for fold in children_folders:

        # Get a list of all the files that are videos or images and save them in a dictionary 
        animal_files  = service.files().list(
            q=f"'{fold["id"]}' in parents  and (mimeType contains 'image/' or mimeType contains 'video/')",
            spaces='drive',
            fields="files(id, name, mimeType)",
            pageSize=1000,
        ).execute()

        # Keys of the dictionary are the folders:
        # random cat, same cat, random dog, same dog
        animal_files_in_folders[fold["name"]] = animal_files.get('files', [])


    # Save here all the pictures as tensors and labels
    tensor_dictionary = {}
    label_dictionary = {}

    # Loop over each folder: random cat, same cat, random dog, same dog
    for animal_fold in list(animal_files_in_folders.keys()):
        tensor_dictionary[animal_fold] = []
        label_dictionary[animal_fold] = []

        # Loop over each file of a folder
        for file in tqdm.tqdm(animal_files_in_folders[animal_fold]):
                        
            file_id = file["id"]

            # If it is an image, process it like an image
            if "image" in file["mimeType"].lower():
    
                # Download the image from Google Drive
                request = service.files().get_media(fileId=file_id)
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                
                fh.seek(0)
                
                # Step 3: Enable HEIF support in Pillow
                register_heif_opener()
                
                # Open image with Pillow
                try:
                    image = Image.open(fh).convert('RGB')                   
                    
                    # Convert to PyTorch tensor
                    transform = transforms.ToTensor()
                    tensor_image = transform(image)
    
                    # Add the image as a tensor to the dictionary of tensors
                    tensor_dictionary[animal_fold].append(tensor_image)
    
                    # Add to the dictionary of labels the label of the animal
                    label_dictionary[animal_fold].append(animal_fold + "_from_image")

                except:
                    print(f"Skipping file {file}: Unidentified image format.")
                    continue
                




            # If it is a video, process it here
            elif "vid" in file["mimeType"].lower():

                # Every n secods take a frame
                n = 2


                request = service.files().get_media(fileId=file_id)
                video_bytes = io.BytesIO()
                downloader = MediaIoBaseDownload(video_bytes, request)
                
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                
                video_bytes.seek(0)
                
                # Save video to disk temporarily
                with open("temp_video.mp4", "wb") as temp_video_file:
                    temp_video_file.write(video_bytes.read())
                
                # Extract frames every n seconds and convert to tensors
                transform = transforms.ToTensor()
                video_capture = cv2.VideoCapture("temp_video.mp4")
                
                fps = video_capture.get(cv2.CAP_PROP_FPS)
                interval = int(fps * n)  # Frame interval for n seconds
                frame_id = 0
                
                while video_capture.isOpened():
                    ret, frame = video_capture.read()
                    if not ret:
                        break
                    if frame_id % interval == 0:
                        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        pil_image = Image.fromarray(rgb_frame)
                        tensor_image = transform(pil_image)
                        
                        # Add the image as a tensor to the dictionary of tensors
                        tensor_dictionary[animal_fold].append(tensor_image)
        
                        # Add to the dictionary of labels the label of the animal
                        label_dictionary[animal_fold].append(animal_fold + "_from_video")
                        
                    frame_id += 1
                
                video_capture.release()

        print(f"Saving {animal_fold} tensors to h5 file.")
        
        with h5py.File(f"assets/raw dataset/pet_sentinel_data_{animal_fold}.h5", 'w') as f:
            for key in tensor_dictionary:
                group = f.create_group(key)
                for i, (tensor, label) in tqdm.tqdm(enumerate(zip(tensor_dictionary[key], label_dictionary[key]))):
                    dset = group.create_dataset(f'tensor_{i}', data=tensor)
                    dset.attrs['label'] = label  # Store label as attribute

        # Delete the tensor list to free space in memory
        del tensor_dictionary[animal_fold] 
        gc.collect()


        


except HttpError as e:
    print("Error: ", str(e))



100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Saving same dog tensors to h5 file.


0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.76s/it]


Saving same cat tensors to h5 file.


1it [00:00,  4.33it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.01s/it]


Saving random cats tensors to h5 file.


1it [00:00,  4.40it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.10s/it]


Saving random dogs tensors to h5 file.


1it [00:00, 407.29it/s]


#### Opening files to see the images

Below code shows how to load h5 files and open the image saved as tensor.

In [7]:
# Load data from a HDF5 file 

tensor_dict = {}
label_dict = {}

with h5py.File('assets/raw dataset/pet_sentinel_data_same cat.h5', 'r') as f:
    for key in f:
        group = f[key]
        tensors = []
        labels = []
        for dname in group:
            dset = group[dname]
            tensors.append(dset[()])
            labels.append(dset.attrs['label'])
        tensor_dict[key] = tensors
        label_dict[key] = labels

In [9]:
tensor_dict.keys(), label_dict.keys()

(dict_keys(['same cat']), dict_keys(['same cat']))

In [19]:

# Image tensor
image_tensor = torch.from_numpy(tensor_dict["same cat"][0])  

# Convert to PIL image
to_pil = transforms.ToPILImage()
image = to_pil(image_tensor)

# Show the image
image.show() 
